# Hierarchical Clustering on Target Sentences

to help deriving a taxonomy for intents, hierarchical clustering is being leveraged.

In [2]:
from bertopic import BERTopic
from scipy.cluster import hierarchy as sch
import pandas as pd
import logging

In [3]:
df =pd.read_parquet('../../data/processed/targets/avocado_train_targets_exploded_cleaned.parquet', engine='fastparquet')

In [4]:
df['target'] = df['target'].str.replace(r'\r\n', '', regex=True)


## Training the Model

In [5]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("BERTopic")
logger.setLevel(logging.INFO)

In [6]:
docs = df['target'].tolist()

In [7]:
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(docs)

2024-10-27 17:44:39,427 - BERTopic - Embedding - Transforming documents to embeddings.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
Batches:  10%|█         | 2214/21858 [03:37<32:06, 10.20it/s]


KeyboardInterrupt: 

## Inspecting clustering results

In [ ]:
# Hierarchical topics
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = topic_model.hierarchical_topics(docs, linkage_function=linkage_function)

In [ ]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)